In [1]:
import itertools
from opencage.geocoder import OpenCageGeocode
from bs4 import BeautifulSoup as bs
import numpy as np
from selenium import webdriver
from splinter import Browser
import pandas as pd
import os
import time
import requests
import warnings
warnings.filterwarnings('ignore')

In [2]:
path = "Aviation_Accident_Report_df2.csv"
aar_csv = pd.read_csv(path)
aar_csv

,Unnamed: 0,Report Number,NTSB Title,Accident Date,Report Date,City,State
0,0,ASR-20-06,Provide Inspectors with Automatic Notification...,6/21/2019,12/16/2020,Mokuleia,HI
1,1,ASR-20-05,F&M Enterprises and Stratus Oil Filter Adapter...,11/30/2020,11/30/2020,Multiple,Multiple
2,2,AAR-20-02,Rapid Descent and Crash into Water Atlas Air I...,2/23/2019,7/14/2020,Trinity Bay,TX
3,3,ASR-20-04,"Install Flight Data, Audio, and Image Recorder...",7/1/2017,5/19/2020,Multiple,Multiple
4,4,AAR-20-01,Helicopter Air Ambulance Collision with Terrai...,1/29/2019,5/19/2020,Zaleski,OH
...,...,...,...,...,...,...,...
475,514,AAR-67-AE,"​Rexall Drug and Chemical Company Lear Jet 23,...",4/12/1966,12/19/1967,Clarendon,TX
476,516,AAR-67-AB,"​Lear Jet Corporation, Lear Jet Model 23, N8041J",10/21/1965,12/11/1967,Jackson,MI
477,517,AAR-67-AF,"​West Coast Airlines, Inc., DC-9, N9101",10/1/1966,12/11/1967,Wemme,OR
478,518,AAR-67-AD,"​Japan Air Lines, Ltd., Douglas DC-8-33, JA-8006",12/25/1965,6/26/1967,San Francisco,CA


In [3]:
cities = aar_csv["City"].to_list()
states = aar_csv["State"].to_list()

key = "8006a222e9ce4db7a69d9e8ba04f10e8"

lati = []
longi = []


for (city, state) in zip(cities, states):
    try:
        geocoder = OpenCageGeocode(key)
        query = f'{city}, {state}'
        results = geocoder.geocode(query)
        lat = results[0]['geometry']['lat']
        lng = results[0]['geometry']['lng']
        lati.append(lat)
        longi.append(lng)
        time.sleep(1)
    except KeyError:
        print(city)


In [4]:

aar_csv['Latitude'] = lati
aar_csv['Longitude'] = longi

In [5]:
aar_csv
aar_csv.to_csv('Aviation_Accident_Report_df2.csv', index = False)


In [6]:
#Just testing stuff.... this is probably useless so ignore it please. Or don't. It's up to you really.

url ='https://aviation-safety.net/database/dblist.php?Year=1971'
#soup = BeautifulSoup(requests.get(the_url).text)
#aar_tables = pd.read_html(str(soup.find_all('table')[9]))
r = requests.get(url)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(r)

NameError: name 'pprint' is not defined

In [ ]:


driver = webdriver.Chrome()
driver.get("https://aviation-safety.net/database/dblist.php?Year=1971")

html = driver.page_source

tables = pd.read_html(html)

driver.close()



In [ ]:
# THE DEED IS DONE AND I GOT THE TABLE

deaths = tables[0]
deaths

In [ ]:

#Url params
pages = [1, 2, 3, 5, 6]
years = np.arange(1971, 2022, 1).tolist()

#List results
Dates = []
Type = []
Registration = []
Operator = []
Fatalities = []
Location = []
Cat = []


for year in years:
    for page in pages:
        try:
            driver = webdriver.Chrome()
            driver.get(f"https://aviation-safety.net/database/dblist.php?Year={year}&lang=&page={page}")
            html = driver.page_source
            a_tables = pd.read_html(html)
            accidents = a_tables[0]
            Dates.append(accidents["date"].to_list())
            Type.append(accidents["type"].to_list())
            Registration.append(accidents["registration"].to_list())
            Operator.append(accidents["operator"].to_list())
            Fatalities.append(accidents["fat."].to_list())
            Location.append(accidents["location"].to_list())
            Cat.append(accidents["cat"].to_list())
            
            driver.close()
        
        except (KeyError, ValueError):
            print("Page not found")
            
            driver.close()
        
        





In [ ]:

new_Dates = list(itertools.chain.from_iterable(Dates))
new_Type = list(itertools.chain.from_iterable(Type))
new_Registration = list(itertools.chain.from_iterable(Registration))
new_Operator = list(itertools.chain.from_iterable(Dates))
new_Fatalities = list(itertools.chain.from_iterable(Fatalities))
new_Location = list(itertools.chain.from_iterable(Location))
new_Cat = list(itertools.chain.from_iterable(Cat))


In [ ]:
mistakes = {'Date': new_Dates,
        'Type': new_Type,
        'Registration': new_Registration,
        'Operator': new_Operator,
        'Fatalities': new_Fatalities,
        'Location': new_Location,
        'Cat': new_Cat
        }

Accident_df = pd.DataFrame(mistakes, columns = ['Date', 'Type', 'Registration',
                                               'Operator', 'Fatalities', 'Location', 'Cat'])

Accident_df

In [ ]:
Accident_df.to_csv('50_year_accident.csv', index = False)